# 신경망
**인공신경망(artificial neural network, 신경망)** 은 뇌의 동작에서 영감을 받은 예측 모델.

뇌는 뉴런의 집합, 각 뉴런은 다른 뉴런의 출력한 결과를 입력 받아 특정 연산을 수행, 계산 결과가 특정 임계치를 넘으면 활성화되고 넘지 않으면 활성화되지 않는다.

인공신경망은 인공적인 뉴런으로 구성, 입력값을 받아 계산을 수행, 신경망은 필기인식이나 얼굴인식 등 다양한 문제를 풀 수 있고 딥러닝에서 활발하게 활용되고 있다.

대부분의 신경망은 **블랙박스(black box)** 이기 때문에 그 속을 들여다본들 어떻게 문제를 풀고 있는지 제대로 이해할 수 없고 큰 신경망은 학습시키기도 어렵다. 언젠가 인공지능을 만들어 기술적 특이점(sigularity)을 앞당기고자 할 때 신경망은 좋은 선택일 수 있다.

## 18.1 퍼렙트론
**퍼셉트론(perceptron)** 은 n개의 이진수(binary)가 하나의 뉴런을 통과해서 가중 합이 0보다 크면 활성화되는 가장 단순한 신경망 구조.

In [1]:
from typing import List, Tuple

Vector = List[float]

def dot(v: Vector, w: Vector) -> float:
    """v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w),  "vectors must be same length"
    
    return sum(v_i * w_i for v_i, w_i in zip(v,w))

In [2]:
def step_function(x: float) -> float:
    return 1.0 if x >= 0 else 0.0

In [3]:
def perceptron_output(weights: Vector, bias: float, x: Vector) -> float:
    """퍼셉트론이 활성화되면 1, 아니면 0을 반환"""
    calculation = dot(weights, x) + bias
    return step_function(calculation)

퍼셉트론은 x를 초평면(hyperplane)으로 구분된 두 개의 공간으로 분리한다. (하나의 뉴런을 통과해서 가중 합이 0보다 크면 활성화 되기 때문에)

dot(weights, x) + bias == 0

가중치인 weights만 잘 선택되면 퍼셉트론으로도 여러 가지 간단한 문제를 풀 수 있다. 

e.g. 

두 개의 입력값이 모두 1이면 1을 반환, 하나라도 1이 아니면 0을 반환하는 'AND 게이트'를 만들 수 있다.

In [4]:
and_weights = [2.,2]
and_bias = -3.

In [5]:
assert perceptron_output(and_weights, and_bias, [1,1]) == 1
assert perceptron_output(and_weights, and_bias, [0,1]) == 0
assert perceptron_output(and_weights, and_bias, [1,0]) == 0
assert perceptron_output(and_weights, and_bias, [0,0]) == 0

두 입력값이 모두 1이면 calculation이 2 + 2 - 3 = 1 이 되어 출력값은 1이다. 입력값 중 하나만 1이면 calculation은 2 + 0 - 3 = -1이 되어 출력값이 0이다. 입력값이 모두 0이면 calculation은 -3이 되어 출력값이 0이 된다. 

비슷한 방식으로 'OR 게이트'도 만들 수 있다.

In [6]:
or_weights = [2., 2]
or_bias = -1.

In [7]:
assert perceptron_output(or_weights, or_bias, [1,1]) == 1
assert perceptron_output(or_weights, or_bias, [0,1]) == 1
assert perceptron_output(or_weights, or_bias, [1,0]) == 1
assert perceptron_output(or_weights, or_bias, [0,0]) == 0

하나의 입력값을 받아 1을 0으로, 0을 1으로 변환하는 'NOT 게이트'도 만들 수 있다.

In [8]:
not_weights = [-2.]
not_bias =1.

In [9]:
assert perceptron_output(not_weights, not_bias, [0]) == 1
assert perceptron_output(not_weights, not_bias, [1]) == 0

위 AND gate는 1,1일 때 perceptron_output의 결과가 0보다 크게 하는 weights와 bias를 설정하면 된다. 

나머지 OR gate, NOT gate도 각각의 조건에 맞게 설정하면 된다.

그러나 단일 퍼셉트론만으로 풀 수 없는 문제도 많다. 

e.g.

아무리 고심해도 단일 퍼셉트론으로 둘 중 하나의 입력값이 1일 때 1을 반환하고 다른 모든 경우에는 0을 반환하는 'XOR 게이트'를 만들 수 없다. 

이런 문제를 풀려면 좀 더 복잡한 신경망이 필요

논리 게이트를 만들기 위해 반드시 뉴련을 사용해야 하는 것은 아니다.

In [10]:
and_gate = min
or_gate = max
xor_gate = lambda x, y: 0 if x == y else 1

우리 뇌의 실제 뉴런들이 그렇듯, 인공 뉴련 역시 서로 연결되었을 때 더욱 흥미로워진다. 

## 18.2 순방향 신경망
뇌의 구조는 매우 복잡하다. 인공신경망으로 뇌를 묘사할 때는 한 방향으로 연결된 **개별 층(layer)으로 추상화**하는 것이 **일반적**이다. 보통은 **입력값으로 받아 그대로 다음 층으로 값을 전송하는 입력층(input layer)**, **하나 이상의 은닉층(hidden layer)** 그리고 **최종값을 반환하는 출력층(output layer)** 등으로 구성한다.

퍼셉트론과 마찬가지로 (입력층에 속하지 않은) 각 뉴런에는 가중치와 편향이 있다. 여기서는 표현을 단순하게 하기 위해 편향을 가중치 벡터의 끝에 덧붙
인 후, 항상 1의 값을 가지는 편향의 입력값을 각 뉴련에 전달하자. 

각 뉴런은 퍼셉트론과 마찬가지로 입력값과 가중치의 곱을 합한다(dot product 수행). 하지만 여기서는 그 값에 step_function을 적용하는 것이 아니라, 조금 더 부드러운 모양을 가진 **시그모이드(sigmoid) 함수**를 적용해 볼 것이다.

In [11]:
import math

def sigmoid(t: float) -> float:
    return 1 / (1 + math.exp(-t))

단순한 step_function 대신 sigmoid를 사용하는 이유?

신경망을 학습하기 위해서는 미적분을 써야하고, 미적분을 사용하기 위해서는 **매끄러운 함수(smooth function)** 를 사용해야 하기 때문이다.

step_function은 연속적인 값을 가지지도 않지만, sigmoid는 그것을 잘 근사한(approximate) 매끄러운 곡선이다.

16장 '로지스틱 회귀 분석'에서 sigmoid 함수를 logistic이라고 불렀다. sigmoid와 logistic은 혼용해서 사용, 기술적으로 'sigmoid'는 함수의 모양을 지칭, 'logistic'은 함수 자체를 가리키는 말이다.

출력값은 아래와 같이 계산할 수 있다.

In [12]:
def neuron_output(weights: Vector, inputs: Vector) -> float:
    # weights에는 편향이 포함되어 있고, inputs는 1을 포함한다.
    return sigmoid(dot(weights, inputs))

이 함수가 주어지면 각 뉴런은 입력값의 수에 1이 추가된 길이의 (가중치 벡터에 편향을 추가했기 때문)벡터로 표현할 수 있다. 그러면 여러 뉴런으로 각 층을 구성하고, 여러 층으로 최종 신경망을 표현하면 된다.

즉, **신경망은 (여러) 리스트의 (뉴런의) 리스트의(가중치의) 벡터** 로 표현할 수 있다.

입력층을 시작으로 출력층의 값이 출력되는 과정을 **순전파(feedforward)** 라고 부른다.

In [13]:
from typing import List

def feed_forward(neural_network: List[List[Vector]],
                input_vector: Vector) -> List[Vector]:
    """
    신경망에 입력 벡터를 전달한다. 
    (가장 마지막뿐 아니라) 모든 층의 결괏값을 반환한다.
    """
    outputs: List[Vector] = []
        
    for layer in neural_network:
        input_with_bias = input_vector + [1]              # 상수를 더한다.
        output = [neuron_output(neuron, input_with_bias)  # 결괏값을 
                 for neuron in layer]                     # 각 뉴런에 대해 계산한다.
        outputs.append(output)                            # 결괏값에 추가한다.
        
        # 이번 층의 결괏값은 다음 층의 입력값이 된다.
        input_vector = output
    return outputs

그럼 이제 단일 퍼셉트론으로는 만들 수 없었던 XOR 게이트를 만들어 보자.

neuron_output이 0 또는 1에 가까운 값을 가질 수 있도록 가중치의 크기를 조금 키워 주면 된다.

In [14]:
xor_network = [# 은닉층
                [[20., 20, -30], # 'and' 뉴런
                [20., 20, -10]], # 'or' 뉴런
               # 출력층
                [[-60., 60, -30]]]  # 1번째 입력값이 아닌 2번째 입력값을 받는 뉴런

In [15]:
# 순방향 신경망은 모든 층에 대한 결과를 계산하기 때문에
# [-1]은 최종결과를, [0]은 해당 벡터에서 값을 반환한다.
assert 0.000 < feed_forward(xor_network, [0, 0])[-1][0] < 0.001
assert 0.999 < feed_forward(xor_network, [1, 0])[-1][0] < 1.000
assert 0.999 < feed_forward(xor_network, [0, 1])[-1][0] < 1.000
assert 0.000 < feed_forward(xor_network, [1, 1])[-1][0] < 0.001

2차원 입력벡터를 받으면 은닉층은 두 입력값의 'and'와 'or'에 해당되는 값으로 구성된 2차원 벡터를 생성한다. 

그리고 출력층은 은닉층에서 받은 2차원 벡터에서 '첫 번째 입력값이 아닌 두 번째 입력값'을 계산해 준다. 그 결과는 'or이지만 and는 아닌' 네트워크, 즉 XOR이 된다. 

In [25]:
# feed_forward(xor_network, [0, 0])
# neural_network == xor_network, input_vector == [0, 0]
# input_with_bias = input_vector + [1] == [0, 0, 1]

neural_network = xor_network = [[[20., 20, -30], 
                                 [20., 20, -10]], 
                                [[-60., 60, -30]]]
input_vector = [0, 0]
outputs: List[Vector] = []

for layer in neural_network:
    print("neural network:", neural_network)
    print("layer:", layer)
    input_with_bias = input_vector + [1]              # 상수를 더한다.
    output = [neuron_output(neuron, input_with_bias)  # 결괏값을 
             for neuron in layer]                     # 각 뉴런에 대해 계산한다.    
    print("output:", output)
    outputs.append(output)                            # 결괏값에 추가한다.

neural network: [[[20.0, 20, -30], [20.0, 20, -10]], [[-60.0, 60, -30]]]
layer: [[20.0, 20, -30], [20.0, 20, -10]]
output: [9.357622968839299e-14, 4.5397868702434395e-05]
neural network: [[[20.0, 20, -30], [20.0, 20, -10]], [[-60.0, 60, -30]]]
layer: [[-60.0, 60, -30]]
output: [9.357622968839299e-14]


2차원 입력벡터를 받으면 은닉층은 두 입력값의 'and'와 'or'에 해당되는 값으로 구성된 2차원 벡터를 생성한다.

그리고 출력층은 은닉층에서 받은 2차원 벡터에서 '첫 번째 입력값이 아닌 두 번째 입력값'을 계산해 준다. 그 결과는 'or 이지만 and는 아닌' 네트워크, 즉 XOR이 된다.

In [24]:
neural_network = xor_network = [[[20., 20, -30], 
                                 [20., 20, -10]], 
                                [[-60., 60, -30]]]
input_vector = [0, 0]
outputs: List[Vector] = []

for layer in neural_network:
    print("neural network:", neural_network)
    print("layer:", layer)
    input_with_bias = input_vector + [1]              # 상수를 더한다.
    for neuron in layer:
        print("neuron:", neuron)
        output = neuron_output(neuron, input_with_bias)
    print("output:", output)
    outputs.append(output)                            # 결괏값에 추가한다.

neural network: [[[20.0, 20, -30], [20.0, 20, -10]], [[-60.0, 60, -30]]]
layer: [[20.0, 20, -30], [20.0, 20, -10]]
neuron: [20.0, 20, -30]
neuron: [20.0, 20, -10]
output: 4.5397868702434395e-05
neural network: [[[20.0, 20, -30], [20.0, 20, -10]], [[-60.0, 60, -30]]]
layer: [[-60.0, 60, -30]]
neuron: [-60.0, 60, -30]
output: 9.357622968839299e-14


**그림 18-3의 선들이 뉴런. 즉, 리스트가 신경망이라고 생각!**,

은닉층은 입력의 특성(feature)을 계산하고 (이 경우 'and'와 'or'), 출력층은 그 특성을 알맞게 조합하는 것으로 해석할 수 있다.

## 18.3 역전파
보통은 앞에서와 같이 신경망을 수동으로 만들지 않는다. 신경망은 훨씬 복잡한 문제를 풀 때 이용되기 때문이기도 하고(예를 들어 이미지 인식은 수백에서 수 천 개의 뉴런을 사용한다), 각 뉴런이 어떤 역할을 하는지 알 수 없기 때문이기도 하다. 

대신 앞에서 했듯 데이터를 이용해서 신경망을 학습시킬 것이다. 이 때 흔히 사용되는 방법으로는 **역전파(backpropagation)** 가 있다.

역전파에는 경사 하강법 또는 그 변이가 사용된다.

입력 벡터와 출력 벡터로 구성된 학습 데이터가 주어졌다고 해보자. 예를 들어 앞의 xor_network 예시에서는 입력 벡터가 [1, 0]일 때 출력 벡터가 [1]이었다. 이제 가중치만 적절히 조정해 주면 되는데, 이는 다음의 알고리즘으로 할 수 있다.

1. 입력 벡터에 대해 feed_forward를 수행하고 모든 뉴런의 출력값을 계산한다.
2. 결괏값을 알기 때문에, 오류의 제곱의 합인 손실(loss)을 계산할 수 있다.
3. 출력층 뉴런의 가중치에 따라 손실의 그래디언트를 계산한다.
4. 은닉층 뉴런의 그래디언트를 계산하기 위해 출력츠의 그래디언트와 오류를 뒤로 '전파'한다.
5. 경사 하강법을 한 번 진행한다.

보통 이 알고리즘이 수렴할 때까지 학습 데이터에 반복적으로 적용한다.

아래는 그래디언트를 계산할 함수이다.

**추가 내용**


In [26]:
def sqerror_gradients(network: List[List[Vector]],
                     input_vector: Vector,
                     target_vector: Vector) -> List[List[Vector]]:
    """
    신경망, 입력 벡터, 출력 벡터가 주어졌다고 가정하고,
    예측값을 출력하고, 가중치에 대한 손실의 제곱으로 
    그래디언트를 계산한다.
    """
    
    # 순방향 전파
    hidden_outputs, outputs = feed_forward(network, input_vector)
    
    # 함수를 적용하기 전 출력층 뉴런의 가중치에 따른 그래디언트
    output_deltas = [output * (1 - output) * (output - target)
                    for output, target in zip(outputs, target_vector)]
    
    # 출력층 뉴런의 가중치에 따른 그래디언트
    output_grads = [[output_deltas[i] * hidden_output
                    for hidden_output in hidden_outputs + [1]]
                   for i, output_neuron in enumerate(network[-1])]
    
    # 함수를 적용하기 전 은닉층 뉴런의 가중치에 따른 그래디언트
    hidden_deltas = [hidden_output * (1 - hidden_output) *
                    dot(output_deltas,[n[i] for n in network[-1]])
                    for i, hidden_output in enumerate(hidden_outputs)]
    
    # 은닉층 뉴런의 가중치에 따른 그래디언트
    hidden_grads = [[hidden_deltas[i] * input for input in input_vector + [1]]
                   for i, hidden_neuron in enumerate(network[0])]
    
    return [hidden_grads, output_grads]

위의 코드를 통해 그래디언트를 계산 할 수 있게 되었다. 앞서 XOR 네트워크를 손으로 계산했다면 이제 위 코드를 통해 진행해보자.

먼저, 학습 데이터를 생성하고, 신경망을 임의의 가중치로 초기화해보자

In [27]:
import random
random.seed(0)

# 학습 데이터
xs = [[0., 0], [0., 1], [1., 0], [1., 1]]
ys = [[0.], [1.], [1.], [0.]]

# 임의의 가중치로 네트워크를 초기화
network = [# 은닉층: 2개 입력 -> 2개 출력
[[random.random() for _ in range(2 + 1)],  # 첫 번째 은닉 뉴런
[random.random() for _ in range(2 + 1)]],  # 두 번째 은닉 뉴런
# 출력층: 2개 입력 -> 1개 출력
[[random.random() for _ in range(2 + 1)]]  # 첫 번째 출력 뉴런
]

경사 하강법으로 신경망을 학습시킬 수 있다. 하지만 이전 예시와 다르게 이번에는 파라미터 벡터가 존재한다. 파라미터 벡터별로 그래디언트가 존재하기 때문에 각각에 대해 gradient_step을 호출해야 한다.

In [31]:
import tqdm

def add(v: Vector, w: Vector) -> Vector:
    """각 성분끼리 더한다."""
    assert len(v) == len(w), "vector must be the same length"
    
    return [v_i + w_i for v_i, w_i in zip(v, w)]

def scalar_multiply(c: float, v: Vector) -> Vector:
    """모든 성분을 c로 곱하기"""
    return [c * v_i for v_i in v]

def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector:
    """v에서 step_size만큼 이동하기"""
    assert len(v) == len(gradient)
    step = scalar_multiply(step_size, gradient)
    return add(v, step)

In [32]:
learning_rate = 1.0
for epoch in tqdm.trange(20000, desc="neural net for xor"):
    for x, y in zip(xs, ys):
        gradients = sqerror_gradients(network, x, y)
        
        # 각 층의 각 뉴런에 대해 gradient step을 취한다.
        network = [[gradient_step(neuron, grad, -learning_rate)
                   for neuron, grad in zip(layer, layer_grad)]
                  for layer, layer_grad in zip(network, gradients)]

neural net for xor: 100%|██████████| 20000/20000 [00:01<00:00, 18694.36it/s]


In [33]:
# XOR을 학습했는지 확인한다.
assert feed_forward(network, [0, 0])[-1][0] < 0.01
assert feed_forward(network, [0, 1])[-1][0] > 0.99
assert feed_forward(network, [1, 0])[-1][0] > 0.99
assert feed_forward(network, [1, 1])[-1][0] < 0.01

In [81]:
print("network:", network)

network: [[[6.953505610104289, 6.952785792366962, -3.1484761965046655], [5.115899442661922, 5.115407875835949, -7.839603434415663]], [[10.961705832630562, -11.63060534664317, -5.144229056613082]]]


## 18.4 예시: Fizz Buzz
유명한 프로그래밍 챌린지인 Fizz Buzz 문제

1 ~ 100까지의 숫자 중에서, 3으로 나눠진다면 "fizz"를 출력하고,

5로 나눠진다면 "buzz"를 출력하고, 15로 나눠진다면 "fizzbuzz"를 출력해라.

신경망은 벡터를 입력 받아 벡터를 출력한다. 하지만 위 문제에서는 정수를 입력 받아 문자열을 출력해야 한다. 즉, 먼저 문제를 벡터 형태의 문제로 변환해야 한다. 결괏값의 종류가 네 가지이기 때문에 0 또는 1의 값을 가질 수 있는 4차원 벡터를 만들면 된다.

In [82]:
def fizz_buzz_encode(x: int) -> Vector:
    if x % 15 == 0:
        return [0, 0, 0, 1]
    elif x % 5 == 0:
        return [0, 0, 1, 0]
    elif x % 3 == 0:
        return [0, 1, 0, 0]
    else:
        return [1, 0, 0, 0]

In [83]:
assert fizz_buzz_encode(2) == [1, 0, 0, 0]
assert fizz_buzz_encode(6) == [0, 1, 0, 0]
assert fizz_buzz_encode(10) == [0, 0, 1, 0]
assert fizz_buzz_encode(30) == [0, 0, 0, 1]

위 함수로 출력 벡터를 생성할 것이다. 여러 이유로 입력 벡터를 단순히 입력값만 담은 1차원 벡터로 표현해서는 안 된다. 한 개의 입력값은 '강도(intensity)'를 나타낸다. 하지만 이 문제에서는 2는 1의 두 배이고, 4는 2의 두 배라는 것은 무의미하다. 게다가 입력이 하나밖에 없다면 은닉층은 좋은 특성을 뽑지 못하게 되어 아마 문제를 풀지 못할 수도 있다.

이런 경우 사용할 수 있는 기법으로 각 숫자를 1과 0으로 구성된 이진수로 표현하는 방법이 있다.

In [84]:
def binary_encode(x: int) -> Vector:
    binary: List[float] = []
    
    for i in range(10):
        binary.append(x % 2)
        x = x // 2
        
    return binary

In [87]:
assert binary_encode(0) == [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
assert binary_encode(1) == [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
assert binary_encode(10) == [0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
assert binary_encode(101) == [1, 0, 1, 0, 0, 1, 1, 0, 0, 0]
assert binary_encode(999) == [1, 1, 1, 0, 0, 1, 1, 1, 1, 1]

1 ~ 100까지 숫자에 대해 결과를 산출하는 것이 목표, 그 숫자들을 이용해서 신경망을 학습하는 것은 편법(오버피팅되기 때문이다).

101 ~ 1023(10개의 이진수로 표현할 수 있는 가장 큰 수)까지의 숫자를 이용해서 학습을 해보자.

In [88]:
xs = [binary_encode(n) for n in range(101, 1024)]
ys = [fizz_buzz_encode(n) for n in range(101, 1024)]

임의의 가중치로 초기화된 신경망을 생성해 보자. 10개의 입력 뉴런과 (10차원 벡터로 입력할 것이기 때문) 4개의 출력 뉴런으로(4차원 벡터로 출력할 것이기 때문) 구성될 것이다. 은닉층은 25개 뉴런으로 구성하되, 차원을 쉽게 바꿀 수 있도록 변수로 만들자.

In [89]:
NUM_HIDDEN = 25

network = [
    # 은닉층: 10개 입력 -> NUM_HIDDEN 출력
    [[random.random() for _ in range(10 + 1)] for _ in range(NUM_HIDDEN)],
    
    # output_layer: NUM_HIDDEN 입력 -> 4개 출력
    [[random.random() for _ in range(NUM_HIDDEN + 1)] for _ in range(4)]
]

학습을 시작해보자. 이것은 약간 더 복잡하기 때문에 학습과정을 면밀하게 살펴보자. 특히 각 에폭(epoch)에서 오차 제곱의 합이 줄어드는지 확인하자

In [93]:
def subtract(v: Vector, w: Vector) -> Vector:
    """각 성분끼리 뺀다."""
    assert len(v) == len(w), "vectors must be the same length"
    
    return [v_i - w_i for v_i, w_i in zip(v, w)]

def dot(v: Vector, w: Vector) -> float:
    """v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w),  "vectors must be same length"
    
    return sum(v_i * w_i for v_i, w_i in zip(v,w))

def sum_of_squares(v: Vector) -> float:
    """v_1 * v_1 + ... v_n * v_n"""
    return dot(v,v)

def squared_distance(v: Vector, w: Vector) -> float:
    """(v_1 - w_1)**2 + ... + (v_n - w_n)**2"""
    return sum_of_squares(subtract(v, w))

In [94]:
learning_rate = 1.0
with tqdm.trange(500) as t:
    for epoch in t:
        epoch_loss = 0.0
        
        for x, y in zip(xs, ys):
            predicted = feed_forward(network, x)[-1]
            epoch_loss += squared_distance(predicted, y)
            gradients = sqerror_gradients(network, x, y)
            
            # 각 측의 각 뉴런에 대해 매 그래디언트만큼 이동
            network = [[gradient_step(neuron, grad, -learning_rate)
                       for neuron, grad in zip(layer, layer_grad)]
                      for layer, layer_grad in zip(network, gradients)]
            
        t.set_description(f"fizz buzz (loss: {epoch:.2f})")

fizz buzz (loss: 499.00): 100%|██████████| 500/500 [01:43<00:00,  4.85it/s]


이 신경망은 4차원 벡터를 출력하지만 우리는 하나의 예측값을 원한다. 리스트 안의 가장 큰 값의 인덱스를 반환하는 argmax로 예측값을 반환하자.

In [98]:
def argmax(xs: list) -> int:
    """최댓값의 인덱스를 반환한다."""
    return max(range(len(xs)), key=lambda i: xs[i])

In [100]:
assert argmax([0, -1]) == 0   # 0번째 항목이 최댓값
assert argmax([-1, 0]) == 1   # 1번째 항목이 최댓값
assert argmax([-1, 10, 5, 20, -3]) == 3   # 3번째 항목이 최댓값

'Fizz Buzz' 문제를 풀어보자

In [101]:
num_correct = 0
for n in range(1, 101):
    x = binary_encode(n)
    predicted = argmax(feed_forward(network, x)[-1])
    actual = argmax(fizz_buzz_encode(n))
    labels = [str(n), "fizz", "buzz", "fizzbuzz"]
    print(n, labels[predicted], labels[actual])
    
    if predicted == actual:
        num_correct += 1
    
    print(num_correct, "/", 100)

1 1 1
1 / 100
2 2 2
2 / 100
3 fizz fizz
3 / 100
4 4 4
4 / 100
5 buzz buzz
5 / 100
6 fizz fizz
6 / 100
7 7 7
7 / 100
8 8 8
8 / 100
9 fizz fizz
9 / 100
10 buzz buzz
10 / 100
11 11 11
11 / 100
12 fizz fizz
12 / 100
13 13 13
13 / 100
14 14 14
14 / 100
15 fizzbuzz fizzbuzz
15 / 100
16 16 16
16 / 100
17 17 17
17 / 100
18 fizz fizz
18 / 100
19 19 19
19 / 100
20 20 buzz
19 / 100
21 fizz fizz
20 / 100
22 22 22
21 / 100
23 23 23
22 / 100
24 fizz fizz
23 / 100
25 buzz buzz
24 / 100
26 26 26
25 / 100
27 fizz fizz
26 / 100
28 28 28
27 / 100
29 29 29
28 / 100
30 fizzbuzz fizzbuzz
29 / 100
31 31 31
30 / 100
32 32 32
31 / 100
33 fizz fizz
32 / 100
34 34 34
33 / 100
35 buzz buzz
34 / 100
36 fizz fizz
35 / 100
37 37 37
36 / 100
38 38 38
37 / 100
39 fizz fizz
38 / 100
40 buzz buzz
39 / 100
41 41 41
40 / 100
42 fizz fizz
41 / 100
43 43 43
42 / 100
44 44 44
43 / 100
45 fizzbuzz fizzbuzz
44 / 100
46 46 46
45 / 100
47 47 47
46 / 100
48 fizz fizz
47 / 100
49 49 49
48 / 100
50 buzz buzz
49 / 100
51 fizz fizz
5

## 18.5 더 공부해 보고 싶다면
"텐셔플로로 Fizz Buzz 풀기"라는 블로그 추천